EVALUATION METRICS

In [1]:
import json
import math
import os
import re
import collections
from collections import defaultdict
from array import array
import numpy as np
import pandas as pd
import sys

In [2]:
def precision_k(y_true,y_score,k=10):
    order=np.argsort(np.asarray(y_score))[::-1]
    y_true= np.take(np.asarray(y_true),order)
    k=min(k,len(y_true)) #handle if k> len(y_true)
    return np.sum(y_true[:k])/k if k>0 else 0.0

In [3]:
def average_precision(y_true,y_score,k=10):
    order=np.argsort(y_score)[::-1]
    y_true=np.take(np.asarray(y_true),order)
    prec_list=[]
    num_relevant=0
    for i in range(min(k,len(order))):
        if y_true[i]==1:
            num_relevant +=1
            prec_list.append(num_relevant/(i+1))
    return np.sum(prec_list)/num_relevant if num_relevant>0 else 0.0

In [4]:
def recall_k(y_true,y_score,k=10):
    order=np.argsort(np.asarray(y_score))[::-1]
    y_true=np.take(np.asarray(y_true),order)
    relevenat=np.sum(y_true[:k])
    total_rel=np.sum(np.asarray(y_true))
    return relevenat/total_rel if total_rel>0 else 0.0

In [5]:
def f1_k(y_true,y_score,k=10):
    prec=precision_k(y_true,y_score,k)
    rec=recall_k(y_true,y_score,k)
    return (2*prec*rec)/(prec+rec) if (prec+rec)>0 else 0.0

In [ ]:
def map_k(dt,k):
    avp = []
    for q in dt["query_id"].unique():  
        curr_data = dt[dt["query_id"] == q]  
        avp.append(average_precision(np.array(curr_data["labels"]), np.array(curr_data["predicted_relevance"]),
                                      k)) 
    return np.sum(avp) / len(avp), avp  

In [ ]:
def rr_k(y_true,y_score,k=10):
    order=np.argsort(np.asarray(y_score))[::-1]
    y_true= np.take(np.asarray(y_true),order)[:k]
    if np.sum(y_true)==0: 
        return 0
    return 1/(np.argmax(y_true)+1)

In [ ]:
def dcg_k(y_true,y_score,k=10):
    order=np.argsort(y_score)[::-1]
    y_true=np.take(y_true,order[:k])
    gain= 2**y_true-1
    discounts=np.log2(np.arange(len(y_true))+2) #+2 is added because log2(1) is 0
    return np.sum(gain/discounts)

In [ ]:
def ndcg_k(y_true,y_score,k=10):
    dcg= dcg_k(y_true,y_score,k)
    idcg=dcg_k(y_true,y_true,k)
    if not idcg:
        return 0
    return round(dcg/idcg,4)
